Automação com pydobot

In [1]:
#pip install pydobot
#pip install oci
#pip install ads
#pip install oracle-ads
#pip install oracle-ads[data]
#pip install oracle-ads[geo]
#pip install --ignore-installed Pillow==9.0.0
#pip install cx_Oracle 
#pip install oracledb
#pip install requests
#pip install pipenv
#pip install shell
#pip install opencv-python


In [2]:

#device.close()

In [3]:
#token OCI Functions
#token=LnDOiyJe9P#:FGEfB04+

In [1]:
from serial.tools import list_ports
import pydobot
import oci
import requests
import json
from oci.config import from_file
from oci.signer import Signer
import time
from datetime import datetime
import cv2
import base64

In [2]:

port = list_ports.comports()[0].device
device = pydobot.Dobot(port=port, verbose=False)



In [16]:
velocity = 200
acceleration = 200
device.speed(velocity, acceleration)
(homeX, homeY, homeZ, r, j1, j2, j3, j4) = device.pose()
print(f'homeX:{homeX} homeY:{homeY} homeZ:{homeZ} j1:{j1} j2:{j2} j3:{j3} j4:{j4}')
print(f'r:{r}')

homeX:154.7767791748047 homeY:-29.821788787841797 homeZ:-15.647041320800781 j1:-10.905881881713867 j2:23.017704010009766 j3:72.118408203125 j4:0.0
r:-10.905881881713867


In [3]:
#Arquivo config para acesso aos serviços OCI
config = from_file("C:\\Users\\rromanin\\Downloads\\OCI\\config")

In [4]:
#Autenticacao, a ser enviada no post para o serviço
auth = Signer(
tenancy=config['tenancy'],
user=config['user'],
fingerprint=config['fingerprint'],
private_key_file_location=config['key_file'],
pass_phrase=config['pass_phrase']
)


In [30]:
def takePhoto():
    cam = cv2.VideoCapture(0)
    result, image = cam.read()
    if result:
        # saving image in local storage
        cv2.imwrite("c:\inspecao.png", image)

        with open("c:\inspecao.png", "rb") as image_file:
            byte_content = image_file.read()
            base64_bytes = base64.b64encode(byte_content)
            encoded_string = base64_bytes.decode('utf-8')
            
    # If captured image is corrupted, moving to else part
    else:
        print("No image detected. Please! try again")

    return encoded_string

In [6]:
def getOrders():
    #Endpoint da function dobot_orders
    endpoint = "https://fzkvp2yhuqa.us-ashburn-1.functions.oci.oraclecloud.com/20181201/functions/ocid1.fnfunc.oc1.iad.aaaaaaaaaey64yu2ddzjh6hhix7haope7e7kvkysjtecwqtj42ztpoi4i4fq/actions/invoke"

    # body a ser enviado no post dobot_orders
    body = {
    'compartmentId': config['tenancy'],  # root compartment
    'name': 'TestUser', #parametro do serviço de exemplo
    'description': 'Created with a raw request'
    }

    #POST dobot_orders
    response = requests.post(endpoint, json=body, auth=auth)
    response.raise_for_status()

    #Retorno do serviço
    return list(response.json().values())[0]

In [41]:
def log_action(tipo_objeto_id, dt_producao_inicio , dt_producao_fim, imagem_url, flg_processamento_ok=1,tipo_falha_id=-2):
    #Endpoint da function dobot_log
    endpoint = "https://fzkvp2yhuqa.us-ashburn-1.functions.oci.oraclecloud.com/20181201/functions/ocid1.fnfunc.oc1.iad.aaaaaaaa2syneibknugvtqktirlzmzg4hq4yn66g2e4xik4sip35ecccbxxa/actions/invoke"
    
    # body a ser enviado no post dobot_log
    body = {
    'compartmentId': config['tenancy'],  # root compartment
    'tipo_objeto_id': tipo_objeto_id, #tipo de objeto produzido
    'flg_processamento_ok': flg_processamento_ok, #produzido sem problemas?
    'tipo_falha_id': tipo_falha_id, #se defeito, qual tipo? se nenhum= -2
    'dt_producao_inicio': dt_producao_inicio, #se defeito, qual tipo? se nenhum= -2
    'dt_producao_fim': dt_producao_fim, #se defeito, qual tipo? se nenhum= -2
    'imagem_url': imagem_url, #se defeito, qual tipo? se nenhum= -2
    'description': 'Created with a raw request'
    }

    #POST dobot_log
    response = requests.post(endpoint, json=body, auth=auth)
    response.raise_for_status()

    #Retorno do serviço
    return response

In [8]:
def checkQualityVisual(encoded_string):
    #Endpoint da function dobot_visual_check
    endpoint = "https://fzkvp2yhuqa.us-ashburn-1.functions.oci.oraclecloud.com/20181201/functions/ocid1.fnfunc.oc1.iad.aaaaaaaa33gdjalbno7robgmp25r25g5jew7xktn45moi2cykeddbt2egciq/actions/invoke"
    
    # body a ser enviado no post dobot_log
    body = {
    'compartmentId': config['tenancy'],  # root compartment
    'encoded_string': encoded_string, 
    'description': 'Created with a raw request'
    }

    #POST dobot_log
    response = requests.post(endpoint, json=body, auth=auth)
    response.raise_for_status()

    #Retorno do serviço
    return response

In [9]:
listQualidadePeca = getOrders()

#Rotaciona garra
homeHead = -15
#Intervalo entre ações
timeCommand = 100


In [56]:
#with open("c:\inspecao.png", "rb") as image_file:
#	byte_content = image_file.read()
#	base64_bytes = base64.b64encode(byte_content)
#	base64_string = base64_bytes.decode('utf-8')

#	#Checagem Visual
#	response = checkQualityVisual(base64_string)
	

In [10]:
def posicaoFoto():
    device.move_to(homeX+80,homeY-35,homeZ+55,homeHead,wait=True)


In [11]:
def abrirGarra():
    device.grip(False)
    device.wait(timeCommand)

In [12]:
def fecharGarra():
    device.grip(True)
    device.wait(timeCommand)

In [13]:
def posicaoSeguranca():
    abrirGarra()
    device.move_to(homeX, homeY, homeZ, homeHead, wait=False)
    device.move_to(homeX,homeY+120,homeZ,homeHead,wait=False)
    device.move_to( homeX+120,homeY+120,homeZ+100,homeHead,wait=False)

In [14]:
def checkRobo():
    posicaoSeguranca()
    device.move_to(homeX+120,homeY,homeZ+100,homeHead,wait=True)
    posicaoFoto()
    takePhoto()
    device.move_to(homeX+120,homeY,homeZ+100,homeHead,wait=True)
    device.move_to(homeX+120,homeY-40,homeZ,homeHead,wait=True)
    fecharGarra() 

In [21]:
checkRobo()


In [43]:
posicaoSeguranca()
tempo = 200

In [44]:
for item in listQualidadePeca:
	#inicio do processo
	time = datetime.now()
	dt_producao_inicio = time.strftime("%Y%m%d%H%M%S")

	#Rotaciona para posicao da esteira
	device.move_to(homeX+120,homeY,homeZ+100,homeHead,wait=True)

	#Posicao de foto
	time = datetime.now()
	print("Instante da foto inicio:",time)
	posicaoFoto()
	encoded_string = takePhoto()
	time = datetime.now()
	#print("Instante da foto fim:",time)
	
	#Checagem Visual
	time = datetime.now()
	#print("Instante da checagem visual inicio:",time)
	response = checkQualityVisual(encoded_string)
	time = datetime.now()
	#print("Instante da checagem visual fim:",time)
	result = json.loads(response.content)

	#Endereco da foto no object storage
	imagemURL = result.get('objetos')[0]
	print(imagemURL)
	print(result.get('objetos'))
	
	isGear = 0
	if 'Gear' in result.get('objetos'):
		isGear = 1

	device.move_to(homeX+120,homeY,homeZ+100,homeHead,wait=True)
	
	#Baixa garra
	device.move_to(homeX+120,homeY-40,homeZ,homeHead,wait=True)

	#Fecha a garra
	fecharGarra()

	#Sobe garra
	device.move_to(homeX+120,homeY-40,homeZ+100,homeHead,wait=True)

	#Verifica se é uma engrenagem
	if isGear > 0 and item == 1:
		#Move para posicao OK 
		device.move_to(homeX-20,homeY-160,homeZ+100,homeHead,wait=True)
		device.move_to(homeX-20,homeY-160,homeZ,homeHead,wait=True)
		abrirGarra()
		device.move_to(homeX-20,homeY-160,homeZ+100,homeHead,wait=True)
		flg_processamento_ok = 1
		tipo_falha_id = -2
	else:
		#Move para posicao NOK
		device.move_to(homeX+120,homeY+120,homeZ+100,homeHead,wait=True)
		device.move_to(homeX+120,homeY+120,homeZ,homeHead,wait=True)
		abrirGarra()
		device.move_to(homeX+120,homeY+120,homeZ+100,homeHead,wait=True)
		flg_processamento_ok = 0
		tipo_falha_id = 1 # objeto incorreto

	device.wait(tempo)
	tempo = tempo + 100
	print("tempo",tempo)
	#fim do processo
	time = datetime.now()
	dt_producao_fim = time.strftime("%Y%m%d%H%M%S")

	log_action(item, dt_producao_inicio , dt_producao_fim, imagemURL, flg_processamento_ok,tipo_falha_id)

posicaoSeguranca()	
device.suck(False)
print("Fim do ciclo")

Instante da foto inicio: 2023-01-05 18:54:44.104868
https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcci5ks1puo/b/Frankie/o/Dobot_20230105215450.png
['https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcci5ks1puo/b/Frankie/o/Dobot_20230105215450.png', 'Gear', 'Auto part', 'Wheel', 'Tire', 'Metal']
tempo 300
Instante da foto inicio: 2023-01-05 18:55:07.685138
https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcci5ks1puo/b/Frankie/o/Dobot_20230105215514.png
['https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcci5ks1puo/b/Frankie/o/Dobot_20230105215514.png', 'Gear', 'Auto part', 'Wheel', 'Cartoon', 'Tire']
tempo 400
Instante da foto inicio: 2023-01-05 18:55:31.100775
https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcci5ks1puo/b/Frankie/o/Dobot_20230105215537.png
['https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcci5ks1puo/b/Frankie/o/Dobot_20230105215537.png', 'Gear', 'Cartoon', 'Button', 'Drawing', 'Owl']
tempo 500
Instante da foto inicio: 2023-01-05 18:55

In [24]:
device.suck(False)

In [202]:
device.close()

In [151]:
listQualidadePeca

[1, 1, 1, 1, 1, 1, 1]